In [1]:
# Import dependencies
from citipy import citipy
import requests
import json
import pandas as pd
import gmaps

# Google developer API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [2]:
# Import Weather.csv
df = pd.read_csv('output/WeatherPy.csv')
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Simple_Date
0,0,Dikson,73.5069,80.5464,12.88,89,95,18.19,RU,1634968857,2021-10-23
1,1,New Norfolk,-42.7826,147.0587,55.65,84,100,1.66,AU,1634969004,2021-10-23
2,2,Slave Lake,55.2834,-114.7690,41.50,56,100,9.66,CA,1634969005,2021-10-23
3,3,Hasaki,35.7333,140.8333,64.80,47,32,19.26,JP,1634968833,2021-10-23
4,4,Aleksandrovsk-Sakhalinskiy,50.8975,142.1561,41.34,73,86,11.30,RU,1634969007,2021-10-23


In [29]:
coordinates = df[["Lat","Lng"]]
humidity = df["Humidity"]

In [3]:
# narrowing down DataFrame to find ideal weather condition
ideal_weather_df = df[(df["Max Temp"] > 70) & (df["Max Temp"] < 80) & (df["Wind Speed"] < 10) & (df["Cloudiness"] == 0)]
ideal_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Simple_Date
161,161,Sao Filipe,14.8961,-24.4956,79.05,70,0,5.26,CV,1634969143,2021-10-23
267,267,Kang,-23.6752,22.7876,71.44,14,0,9.57,BW,1634969044,2021-10-23
315,315,Adrar,27.8743,-0.2939,70.70,45,0,7.40,DZ,1634968835,2021-10-23
418,418,Rio Branco,-9.9747,-67.8100,79.41,89,0,6.91,BR,1634969216,2021-10-23
458,458,Taoudenni,22.6783,-3.9836,74.88,30,0,6.17,ML,1634969417,2021-10-23
482,482,Dekar,-21.5333,21.9333,75.56,12,0,9.13,BW,1634969440,2021-10-23
502,502,Vao,-22.6667,167.4833,73.63,79,0,7.56,NC,1634969460,2021-10-23
525,525,Agadez,19.7500,10.2500,76.57,19,0,8.05,NE,1634969483,2021-10-23
553,553,Asansol,23.6833,86.9833,79.70,45,0,3.85,IN,1634969511,2021-10-23


In [81]:
# creating a list of lat and lng for request loop
target_coordinates = ideal_weather_df["Lat"].astype(str) + "%2C" + ideal_weather_df["Lng"].astype(str)

# create empty lists for for loop
name = []
city = []
hotel_lat = []
hotel_lng = []

for i in target_coordinates:
    response = requests.get(f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={i}&radius=5000&type=hotel&key={g_key}').json()
    try:
        name.append(json.dumps(response["results"][1]["name"]))
        city.append(json.dumps(response["results"][1]["vicinity"]))
        hotel_lat.append(json.dumps(response["results"][1]["geometry"]["location"]["lat"]))
        hotel_lng.append(json.dumps(response["results"][1]["geometry"]["location"]["lng"]))
    except:
        pass

# create hotel_df for heatmap
hotel_df = pd.DataFrame({
    "name": name,
    "city": city,
    "hotel_lat": hotel_lat,
    "hotel_lng": hotel_lng
})


,name,city,hotel_lat,hotel_lng
0,"""Hotel Xaguate""","""Sao Filipe, Rua da Posto, Fogo""",14.8995932,-24.4981834
1,"""Kang Ultra Stop""","""11015 Kazungula, Gaborone""",-23.6817073,22.7665233
2,"""Air Algerie Agence d'Adrar""","""VPF7+257, Adrar""",27.8725367,-0.2870768
3,"""Inacio's Restaurante""","""Rua Rui Barbosa, 81 - Centro, Rio Branco""",-9.973301799999998,-67.8114258
4,"""Ruines du fort de Taoud\u00e9nit""","""M2G9+G23, Taoudenni""",22.6762648,-3.982448099999999
5,"""Farming""","""Dekar""",-21.5237263,21.9373445
6,"""Hotel Kou-Bugny""","""Kuto""",-22.659695,167.443914
7,"""Indira Gandhi National Open University""","""B B College, Road, Ushagram, Asansol""",23.6811431,86.99754089999999


In [83]:
hotel_df.info[[]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       8 non-null      object
 1   city       8 non-null      object
 2   hotel_lat  8 non-null      object
 3   hotel_lng  8 non-null      object
dtypes: object(4)
memory usage: 384.0+ bytes


In [69]:
# geocoordinates
target_coordinates = "14.8961,-24.4956"
target_type = "hotel"
target_radius = 5000

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a parameters dictionary
params = {
    "location": "14.8961,-24.4956",
    "radius": target_radius,
    "type" : target_type,
    "key" : g_key
}


# run request
response = requests.get(base_url, params=params).json()
#print(json.dumps(response, indent = 4, sort_keys=True))
print(json.dumps(response["results"][1]["geometry"]["location"]))
#print(json.dumps(response["results"][1]["geometry"]["location"]["lng"]))
#print(json.dumps(response["results"][1]))

{"lat": 14.8995932, "lng": -24.4981834}


In [68]:
#print(f'{base_url}location={target_coordinates},type="hotel",key={g_key}, radius=5000')
f'{hotel_lat},{hotel_lng}'

"['14.8995932', '-23.6817073', '27.8725367', '-9.973301799999998', '22.6762648', '-21.5237263', '-22.659695', '23.6811431'],['-24.4981834', '22.7665233', '-0.2870768', '-67.8114258', '-3.982448099999999', '21.9373445', '167.443914', '86.99754089999999']"

In [143]:
ideal_weather_df.iloc[0,8]

'CV'

In [30]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px'
}
fig = gmaps.figure(layout=figure_layout)

# Create heatmap
heatmap_layer = gmaps.heatmap_layer(locations = coordinates,weights = humidity)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='600px', width='800px'))

In [86]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px'
}
fig = gmaps.figure(layout=figure_layout)
heatmap_layer = gmaps.heatmap_layer(locations = coordinates,weights = humidity)
marker_locations = hotel_df[["hotel_lat","hotel_lng"]].astype(float)
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(heatmap_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='600px', width='800px'))

In [77]:
ideal_weather_df[["Lat", "Lng"]]

,Lat,Lng
161,14.8961,-24.4956
267,-23.6752,22.7876
315,27.8743,-0.2939
418,-9.9747,-67.8100
458,22.6783,-3.9836
482,-21.5333,21.9333
502,-22.6667,167.4833
525,19.7500,10.2500
553,23.6833,86.9833
